# The analysis of multiscale model results: insights into a single file

This notebook shows various analyses of the results of the multiscale model. We go into the details of the pulse shaping, the plasma profile, and we show also the build up of the harmonic signal.

See complementary analyses:
* [Jupyter notebook processing a list of results.](./analyse_cell_list.ipynb)
* [Jupyter notebook analysing a file, which contains also the outputs of TDSE module](../density_profile/analyse_density_profile.ipynb) (these results were removed in the actual tutorial because they are the largest datasets from the model).



## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import copy
import itertools
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG

anims_author = 'Jan Vábek'

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


%matplotlib inline

Here we dicide if we `show` animations here or `save` them as well.

In [ ]:
visualisation = 'show'

## Load data

We load the data from the pulse propagation in the Pythonic data container defined [in this module](../../CUPRAD/python/dataformat_CUPRAD.py). It contains the data about the pulse propagation and some further characteristics. The data from the harmonic signal will be loaded later.

In [ ]:
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'gas_cell')

h5file1 = os.path.join(demos_path, 'results_cell_2.h5')

with h5py.File(h5file1,'r') as f:

    # load cuprad data = pulse propagation
    CUPRAD_res = dfC.get_data(f)
    CUPRAD_res.get_plasma(f)


Here we print some basic characteristics of the simulation.

In [ ]:
# code to generate the following text
display(Markdown(
rf"""* The ($1/\mathrm{{e}}$) entry pulse duration ${
      CUPRAD_res.pulse_duration_entry   
      :.1f}~\mathrm{{fs}}$
* The box size is ($z-$ and $\rho$-grids start at 0):
    * $z_{{\mathrm{{max}}}}={
      1e3*CUPRAD_res.zgrid[-1]   
      :.1f}~\mathrm{{mm}},~N_z={
      np.shape(CUPRAD_res.E_zrt)[0]   
      :.0f}$,
    * $t_{{\mathrm{{min/max}}}}=\mp{
      1e15*CUPRAD_res.tgrid[-1]   
      :.0f}~\mathrm{{fs}},~N_t={
      np.shape(CUPRAD_res.E_zrt)[2]   
      :.0f}$,
    * $\rho_{{\mathrm{{max}}}}={
      1e6*CUPRAD_res.rgrid[-1]   
      :.0f}~\mu\mathrm{{m}},~N_\rho={
      np.shape(CUPRAD_res.E_zrt)[1]   
      :.0f}.$
"""))

## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Aside we plot the plasma density (measured in % relative to the particle density).

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pythonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data `CUPRAD_res.vacuum_shift()` or just by sychronising the clocks in the $t$-grid `CUPRAD_res.co_moving_t_grid(zgrid)`).$^\dagger$

$^\dagger$ Note that the shift `CUPRAD_res.vacuum_shift()` is retrieved as the Fourier shift, which imposes periodic conditions. It thus requires vanishing field close to boundaries and cannot be applied to the plasma density. `CUPRAD_res.co_moving_t_grid(zgrid)` is a function and it transforms any `zgrid`.

In [ ]:
tlim = np.asarray((-40,40))   # [fs]  the time range for plotting the propagating pulse
rlim = 200                    # [mum] the radial range for plotting the propagating pulse

z2 = 5e-3

ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'gas_cell', 'export')
if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

In [ ]:
k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)
k_z2             = mn.FindInterval(CUPRAD_res.zgrid ,z2)



fig, axs = plt.subplots(2, 2, figsize=(12, 6),
                        sharey='row',
                        gridspec_kw={'wspace': 0.05, 'hspace': 0.25})

# --- First row: Harmonic cutoff ---
r_grid, sym_data = mn.symmetrize_y(
    1e6 * CUPRAD_res.rgrid[:k_r_max],
    (
        HHG.ComputeCutoff(
            mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0, :k_r_max, k_t_min:k_t_max]) / units.INTENSITYau,
            mn.ConvertPhoton(CUPRAD_res.omega0, 'omegaSI', 'omegaau'),
            mn.ConvertPhoton(CUPRAD_res.Ip_eV, 'eV', 'omegaau')
        )[1]
    ).T
)
pc1 = axs[0, 0].pcolormesh(1e15 * CUPRAD_res.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')

sym_data = mn.symmetrize_y(
    1e6 * CUPRAD_res.rgrid[:k_r_max],
    (
        HHG.ComputeCutoff(
            mn.FieldToIntensitySI(CUPRAD_res.E_zrt[k_z2, :k_r_max, k_t_min:k_t_max]) / units.INTENSITYau,
            mn.ConvertPhoton(CUPRAD_res.omega0, 'omegaSI', 'omegaau'),
            mn.ConvertPhoton(CUPRAD_res.Ip_eV, 'eV', 'omegaau')
        )[1]
    ).T
)[1]
pc2 = axs[0, 1].pcolormesh(1e15 * CUPRAD_res.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')
pc2.set_clim(*pc1.get_clim())

# Shared colorbar for first row
cbar1 = fig.colorbar(pc1, ax=axs[0, :], orientation='vertical', fraction=0.02, pad=0.02)
cbar1.set_label(r'Intensity [harmonic cut-off]', rotation=90)




# --- Second row: Relative plasma density ---
r_grid, sym_data = mn.symmetrize_y(
    1e6 * CUPRAD_res.plasma.rgrid[:k_r_max],
    (
        1e2 / CUPRAD_res.effective_neutral_particle_density *
        CUPRAD_res.plasma.value_zrt[0, :k_r_max, k_t_min:k_t_max]
    ).T
)
pc3 = axs[1, 0].pcolormesh(1e15 * CUPRAD_res.plasma.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')

sym_data = mn.symmetrize_y(
    1e6 * CUPRAD_res.plasma.rgrid[:k_r_max],
    (
        1e2 / CUPRAD_res.effective_neutral_particle_density *
        CUPRAD_res.plasma.value_zrt[k_z2, :k_r_max, k_t_min:k_t_max]
    ).T
)[1]
pc4 = axs[1, 1].pcolormesh(1e15 * CUPRAD_res.plasma.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')
pc4.set_clim(*pc3.get_clim())

# Shared colorbar for second row
cbar2 = fig.colorbar(pc3, ax=axs[1, :], orientation='vertical', fraction=0.02, pad=0.02)
cbar2.set_label(r'Relative plasma density [%]', rotation=90)

# --- Common axis formatting using itertools ---
for ax in itertools.chain(*axs):
    ax.set_xlim(tlim)
    ax.set_ylim((-rlim, rlim))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')

# Only left column gets y-axis label to avoid redundancy
axs[0][0].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')
axs[1][0].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

fig.tight_layout()
plt.show()

## XUV camera

Here we show the far-field XUV spectra together with the build-up of the signal in the generating medium.$^\dagger$

These data might be very large. Please specify here the parametes of the plot, so only the necessary subset of data is loaded and processed.

$^\dagger$ Note that the maximum of the harmonic signal is obttained simply as $I_H(z) = \mathrm{max}_{\omega_0 \xi \in \left[H-\Delta, H+\Delta H \right]}|\mathcal{E}_{\text{XUV, cumulative}}(z,\cdot,\xi)|^2$, we define below $\Delta H$=`delta_H`. One might consider also other metrics as the total energy, etc.

In [ ]:
rmax = 0.007                # [m]    the radial dimesion to read the data
XUV_theta_range = [-4, 4]   # [mrad] the divergence angle for plotting 

# Hmax_plot_linear = 50       # the maximal frequency in the linear plot of the spatially resolved harmonic spectra


kz_step = 10    #       the step in $z$ for plotting (derived from the spacing used in the computational grid)

multipliers = [1,4,10,150]                  # multipliers applied to the build-up to fit the figure nicely
delta_H = 1.                                # the camera spectral range to analyse signal of the harmoonics of the interest

In [ ]:
# import data & basic analyses
with h5py.File(h5file1,'r') as f1:
    # load Hankel data = XUV camera
    ogrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/ogrid'][:]
    rgrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/rgrid'][:]
    zgrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/zgrid'][0:-1:kz_step]

    camera_distance = mn.readscalardataset(f1,MMA.paths['Hankel_inputs']+'/distance_FF','N')
    theta_grid_Hankel = np.arctan(rgrid_Hankel/camera_distance)  # recompute the radial grid to the divergence

    Hgrid_Hankel = ogrid_Hankel/CUPRAD_res.omega0

    kr_max = mn.FindInterval(rgrid_Hankel, rmax) + 1
    rgrid_Hankel_full = copy.copy(rgrid_Hankel)
    rgrid_Hankel = rgrid_Hankel[:kr_max]
    theta_grid_Hankel = theta_grid_Hankel[:kr_max]


    cumulative_field =    f1[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,0] +\
                    1j*f1[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,1] 
    
    entry_plane_transform = f1[MMA.paths['Hankel_outputs']+'/entry_plane_transform'][:kr_max,:,0] +\
                    1j*f1[MMA.paths['Hankel_outputs']+'/entry_plane_transform'][:kr_max,:,1]

    exit_plane_transform = f1[MMA.paths['Hankel_outputs']+'/exit_plane_transform'][:kr_max,:,0] +\
                1j*f1[MMA.paths['Hankel_outputs']+'/exit_plane_transform'][:kr_max,:,1]


## Harmonic spectra 4 cases

In [ ]:
k_z2 = mn.FindInterval(zgrid_Hankel ,z2)

Hrange = (20,50)
theta_range = (-5,5)
orders_to_plot = 3          # the range of the logarithmic plot of the spatially resolved harmonic spectra
fig, axs = plt.subplots(2, 2, figsize=(12, 6),
                        sharey='row',
                        gridspec_kw={'wspace': 0.05, 'hspace': 0.25})

# fig, axs = plt.subplots(1, 2, figsize=(12, 3)) #,
#                         # sharey='row',
#                         # gridspec_kw={'wspace': 0.05, 'hspace': 0.25})



theta_grid_Hankel = theta_grid_Hankel[:kr_max]
# theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[0,:,:]).T)
# data_max = np.max([np.max(np.abs(plane)) for plane in [entry_plane_transform, exit_plane_transform]]) 
data_max = np.max(np.abs(entry_plane_transform))



theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(entry_plane_transform).T/data_max)

# pc1 = axs[0].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
pc1 = axs[0,0].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))
cbar1 = fig.colorbar(pc1, ax=axs[0,0], orientation='vertical') # fraction=0.02, pad=0.02)
cbar1.set_label(r'XUV signal [arb. u.]', rotation=90)

sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(exit_plane_transform).T/data_max)[1]
pc2 = axs[0,1].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))
cbar2 = fig.colorbar(pc2, ax=axs[0,1], orientation='vertical') # fraction=0.02, pad=0.02)
cbar2.set_label(r'XUV signal [arb. u.]', rotation=90)

sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[-1,:,:]).T)[1]
pc3 = axs[1,0].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))
cbar3 = fig.colorbar(pc3, ax=axs[1,0], orientation='vertical') # fraction=0.02, pad=0.02)
cbar3.set_label(r'XUV signal [arb. u.]', rotation=90)

# sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[-1,:,:]).T)[1]
pc4 = axs[1,1].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
cbar4 = fig.colorbar(pc4, ax=axs[1,1], orientation='vertical') # fraction=0.02, pad=0.02)
cbar4.set_label(r'XUV signal [arb. u.]', rotation=90)

for ax in [axs[1,0],axs[1,1]]:
    ax.set_xlim(Hrange)
    ax.set_xlabel(r'Harmonic order [-]')
    
for ax in [axs[0,0],axs[1,0]]:
    ax.set_ylim(theta_range)
    ax.set_ylabel(r'Divergence [mrad]')

# H_min, H_max = axs[0].get_xlim() 
# odd_ticks_major = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10) # Generate odd ticks within the H-range
# odd_ticks_minor = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2)  # Generate odd ticks within the H-range
# for ax in axs:
#     ax.set_xlabel('harmonic order [-]')
#     ax.set_xticks(odd_ticks_major)
#     ax.set_xticks(odd_ticks_minor, minor = True)

corner_x, corner_y = 0.98, 0.95  # upper-right in Axes-fraction coords (0–1)

axs[0, 0].text(corner_x, corner_y, 'a)', transform=axs[0, 0].transAxes, fontsize=16, va='top', ha='right', color='white')
axs[0, 1].text(corner_x, corner_y, 'b)', transform=axs[0, 1].transAxes, fontsize=16, va='top', ha='right', color='white')
axs[1, 0].text(corner_x, corner_y, 'c)', transform=axs[1, 0].transAxes, fontsize=16, va='top', ha='right', color='white')
axs[1, 1].text(corner_x, corner_y, 'd)', transform=axs[1, 1].transAxes, fontsize=16, va='top', ha='right', color='white')


plt.show()


## 